In [1]:
import kerastuner as kt
from tensorflow import keras
import tensorflow as tf
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter as hp
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten
from keras.models import Sequential 
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19
from keras import layers
from keras.preprocessing import image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import datetime
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import itertools
import cv2
from sklearn.metrics import precision_recall_fscore_support as score
from tensorflow.keras.models import Model
!pip install imutils
import imutils
import matplotlib as mpl
import time
from scipy import interp
from sklearn.metrics import roc_curve,auc
from sklearn import metrics
from tensorflow.keras.applications.inception_v3 import preprocess_input
import math
import shutil
!pip install openpyxl

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_28/2424953743.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25838 sha256=b8e8503d15f91b23a96b1b9b64d5c4a8c2307ffc7962ee6c4aa88c0697126b89
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


In [2]:
N_SPLITS=5
FOLDNO=1
IMG_SIZE=256
CHANNELS=3
LEARNING_RATE=1e-6
BATCH_SIZE=32
CLASSES=2
CLASS1="Covid-19"
CLASS2="Normal"
EPOCHS=20
CLASS_MODE='categorical'
CROSSENTROPY='categorical_crossentropy'
MODEL_NAME="DenseNet121"
OPTIMIZER="Adam"
ACTIVATION="softmax"
WEIGHTS="imagenet"
OPTIMIZER_HYPERPARAMS=False
BETA_1=0.9
BETA_2=0.999
EPSILON=0.1
DECAY=0.0
ANNEALER=False
ANNEALER_MONITOR='val_loss'
ANNEALER_FACTOR=0.1
ANNEALER_PATIENCE=3
EARLY=False
EARLY_MONITOR='val_loss'
EARLY_PATIENCE=10
MIN_DELTA=0.001
RESCALE=True
SAMPLEWISE_CENTER=False
SAMPLEWISE_STD_NORM=False
AUGMENTATION=True
ROTATION_RANGE=15
WIDTH_SHIFT_RANGE=0.15
HEIGHT_SHIFT_RANGE=0.15
ZOOM_RANGE=None
HORIZONTAL_FLIP=True
VERTICAL_FLIP=False
TEST_SIZE=0.1666
FINE_TUNING=False
UNFROZEN_LAYERS=0
POOLING='None'
SEED=42
NORMALIZED='Normalized'
SPLIT='NoSplit'
SEGMENTED='NotSegmented'

In [3]:
train_df = pd.read_csv('/kaggle/input/train-covidxct3a-removed/train_COVIDx_CT-3A.txt', sep=" ", header=None)
train_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']
train_df=train_df.drop(['xmin', 'ymin','xmax', 'ymax'], axis=1 )

val_df = pd.read_csv('/kaggle/input/val-covidxct3a-removed/val_COVIDx_CT-3A.txt', sep=" ", header=None)
val_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']
val_df=val_df.drop(['xmin', 'ymin','xmax', 'ymax'], axis=1 )

test_df = pd.read_csv('/kaggle/input/test-covidxct3a-removed/test_COVIDx_CT-3A.txt', sep=" ", header=None)
test_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']
test_df=test_df.drop(['xmin', 'ymin','xmax', 'ymax'], axis=1 )

In [4]:
train=pd.concat([train_df,val_df,test_df])

In [5]:
train_normal=train[train['filename'].str.startswith('COVIDCTMD-normal')]

In [6]:
normal=train_normal['filename'].str.split('-',n=2, expand=True)

In [7]:
normal[0]=normal[0]+'-'+normal[1]

In [8]:
normal=normal.drop(columns=[1])

In [9]:
normal.rename(columns={0: 'patient_id', 2: 'images'}, inplace=True)

In [10]:
normal['target']=CLASS2

In [11]:
normal=normal.drop(columns=['images'])

In [12]:
normal['images']=train_normal['filename']

In [13]:
covid_dir1='/kaggle/input/covidctmd-covid-part1-final/covidctmd_covid_part1/'
covid_dir2='/kaggle/input/covidctmd-covid-part2-final/'
covid_dir3='/kaggle/input/covidctmd-covid-part3/'
covidlist1=os.listdir(covid_dir1)
covidlist2=os.listdir(covid_dir2)
covidlist3=os.listdir(covid_dir3)
covidlist=covidlist1+covidlist2+covidlist3
covidlist.sort()

In [14]:
normal_dir1='/kaggle/input/covidctmd-normal-part1/'
normal_dir2='/kaggle/input/covidctmd-normal-part2/'
normal_dir3='/kaggle/input/covidctmd-normal-part3/'
normallist1=os.listdir(normal_dir1)
normallist2=os.listdir(normal_dir2)
normallist3=os.listdir(normal_dir3)
normallist=normallist1+normallist2+normallist3
normallist.sort()

In [15]:
covid_df1=pd.DataFrame({'images': covidlist1, 'target': CLASS1})
covid_df1['image_path'] = covid_dir1 + covid_df1['images']
covid_df2=pd.DataFrame({'images': covidlist2, 'target': CLASS1})
covid_df2['image_path'] = covid_dir2 + covid_df2['images']
covid_df3=pd.DataFrame({'images': covidlist3, 'target': CLASS1})
covid_df3['image_path'] = covid_dir3 + covid_df3['images']
covid_df=pd.concat([covid_df1, covid_df2, covid_df3])
covid_patient_df=pd.DataFrame()
covid_patient_df['patient_id']=covid_df.images.str.rsplit("-", n=1, expand = True)[0]
final_covid_df=pd.concat([covid_patient_df, covid_df], axis=1)
#covid_pat_unique=pd.DataFrame(final_covid_df['patient_id'].unique())

In [16]:
normal_df1=pd.DataFrame({'images': normallist1, 'target': CLASS2})
normal_df1['image_path'] = normal_dir1 + normal_df1['images']
normal_df2=pd.DataFrame({'images': normallist2, 'target': CLASS2})
normal_df2['image_path'] = normal_dir2 + normal_df2['images']
normal_df3=pd.DataFrame({'images': normallist3, 'target': CLASS2})
normal_df3['image_path'] = normal_dir3 + normal_df3['images']
normal_df=pd.concat([normal_df1, normal_df2, normal_df3])
normal_patient_df=pd.DataFrame()
normal_patient_df['patient_id']=normal_df.images.str.rsplit("_", n=1, expand = True)[0]
final_normal_df=pd.concat([normal_patient_df, normal_df], axis=1)
#normal_pat_unique=pd.DataFrame(final_normal_df['patient_id'].unique())

In [17]:
final_normal_df['images']=final_normal_df['images'].str.replace(r'G', '', regex=True)
final_normal_df['images']=final_normal_df['images'].str.replace(r'_', '-', regex=True)
normal['images']=normal['images'].str[:-3]+'tif'

In [18]:
new_normal_df=final_normal_df[final_normal_df['images'].isin(normal['images'])]

In [19]:
final_covid_df

,patient_id,images,target,image_path
0,COVIDCTMD-P029,COVIDCTMD-P029-IM0071.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
1,COVIDCTMD-P019,COVIDCTMD-P019-IM0002.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
2,COVIDCTMD-P035,COVIDCTMD-P035-IM0016.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
3,COVIDCTMD-P021,COVIDCTMD-P021-IM0012.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
4,COVIDCTMD-P034,COVIDCTMD-P034-IM0114.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
...,...,...,...,...
2158,COVIDCTMD-P122,COVIDCTMD-P122-IM0034.tif,Covid-19,/kaggle/input/covidctmd-covid-part3/COVIDCTMD-...
2159,COVIDCTMD-P165,COVIDCTMD-P165-IM0056.tif,Covid-19,/kaggle/input/covidctmd-covid-part3/COVIDCTMD-...
2160,COVIDCTMD-P138,COVIDCTMD-P138-IM0068.tif,Covid-19,/kaggle/input/covidctmd-covid-part3/COVIDCTMD-...
2161,COVIDCTMD-P122,COVIDCTMD-P122-IM0005.tif,Covid-19,/kaggle/input/covidctmd-covid-part3/COVIDCTMD-...


In [20]:
new_normal_df

,patient_id,images,target,image_path
0,COVIDCTMD-normal025,COVIDCTMD-normal025-IM0015.tif,Normal,/kaggle/input/covidctmd-normal-part1/COVIDCTMD...
1,COVIDCTMD-normal022,COVIDCTMD-normal022-IM0079.tif,Normal,/kaggle/input/covidctmd-normal-part1/COVIDCTMD...
2,COVIDCTMD-normal002,COVIDCTMD-normal002-IM0037.tif,Normal,/kaggle/input/covidctmd-normal-part1/COVIDCTMD...
3,COVIDCTMD-normal018,COVIDCTMD-normal018-IM0027.tif,Normal,/kaggle/input/covidctmd-normal-part1/COVIDCTMD...
4,COVIDCTMD-normal003,COVIDCTMD-normal003-IM0031.tif,Normal,/kaggle/input/covidctmd-normal-part1/COVIDCTMD...
...,...,...,...,...
3635,COVIDCTMD-normal071,COVIDCTMD-normal071-IM0070.tif,Normal,/kaggle/input/covidctmd-normal-part3/COVIDCTMD...
3636,COVIDCTMD-normal055,COVIDCTMD-normal055-IM0087.tif,Normal,/kaggle/input/covidctmd-normal-part3/COVIDCTMD...
3637,COVIDCTMD-normal063,COVIDCTMD-normal063-IM0095.tif,Normal,/kaggle/input/covidctmd-normal-part3/COVIDCTMD...
3639,COVIDCTMD-normal066,COVIDCTMD-normal066-IM0148.tif,Normal,/kaggle/input/covidctmd-normal-part3/COVIDCTMD...


In [21]:
final_normal_df=new_normal_df

In [22]:
train_covid_folds=[]
valid_covid_folds=[]

skf = KFold(n_splits=N_SPLITS,shuffle=True,random_state=SEED)

for train_index, val_index in skf.split(final_covid_df['images']):
    
    training_data = final_covid_df.iloc[train_index]
    valid_data = final_covid_df.iloc[val_index]
    
    train_covid_folds.append(training_data)
    valid_covid_folds.append(valid_data)

In [23]:
train_normal_folds=[]
valid_normal_folds=[]

skf = KFold(n_splits=N_SPLITS,shuffle=True,random_state=SEED)

for train_index, val_index in skf.split(final_normal_df['images']):
    
    training_data = final_normal_df.iloc[train_index]
    valid_data = final_normal_df.iloc[val_index]
    
    train_normal_folds.append(training_data)
    valid_normal_folds.append(valid_data)

In [27]:
train_covid_folds[0]

,patient_id,images,target,image_path
1,COVIDCTMD-P019,COVIDCTMD-P019-IM0002.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
2,COVIDCTMD-P035,COVIDCTMD-P035-IM0016.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
3,COVIDCTMD-P021,COVIDCTMD-P021-IM0012.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
4,COVIDCTMD-P034,COVIDCTMD-P034-IM0114.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
5,COVIDCTMD-P022,COVIDCTMD-P022-IM0061.tif,Covid-19,/kaggle/input/covidctmd-covid-part1-final/covi...
...,...,...,...,...
2155,COVIDCTMD-P148,COVIDCTMD-P148-IM0039.tif,Covid-19,/kaggle/input/covidctmd-covid-part3/COVIDCTMD-...
2157,COVIDCTMD-P166,COVIDCTMD-P166-IM0074.tif,Covid-19,/kaggle/input/covidctmd-covid-part3/COVIDCTMD-...
2158,COVIDCTMD-P122,COVIDCTMD-P122-IM0034.tif,Covid-19,/kaggle/input/covidctmd-covid-part3/COVIDCTMD-...
2160,COVIDCTMD-P138,COVIDCTMD-P138-IM0068.tif,Covid-19,/kaggle/input/covidctmd-covid-part3/COVIDCTMD-...


In [28]:
new_train_covid_fold1=train_covid_folds[0]
new_train_covid_fold2=train_covid_folds[1]
new_train_covid_fold3=train_covid_folds[2]
new_train_covid_fold4=train_covid_folds[3]
new_train_covid_fold5=train_covid_folds[4]
#new_train_covid_fold6=covid[covid['patient_id'].isin(train_covid_folds[5][0])]
#new_train_covid_fold7=covid[covid['patient_id'].isin(train_covid_folds[6][0])]
#new_train_covid_fold8=covid[covid['patient_id'].isin(train_covid_folds[7][0])]
#new_train_covid_fold9=covid[covid['patient_id'].isin(train_covid_folds[8][0])]
#new_train_covid_fold10=covid[covid['patient_id'].isin(train_covid_folds[9][0])]

In [29]:
new_train_normal_fold1=train_normal_folds[0]
new_train_normal_fold2=train_normal_folds[1]
new_train_normal_fold3=train_normal_folds[2]
new_train_normal_fold4=train_normal_folds[3]
new_train_normal_fold5=train_normal_folds[4]
#new_train_covid_fold6=covid[covid['patient_id'].isin(train_covid_folds[5][0])]
#new_train_covid_fold7=covid[covid['patient_id'].isin(train_covid_folds[6][0])]
#new_train_covid_fold8=covid[covid['patient_id'].isin(train_covid_folds[7][0])]
#new_train_covid_fold9=covid[covid['patient_id'].isin(train_covid_folds[8][0])]
#new_train_covid_fold10=covid[covid['patient_id'].isin(train_covid_folds[9][0])]

In [30]:
new_valid_covid_fold1=valid_covid_folds[0]
new_valid_covid_fold2=valid_covid_folds[1]
new_valid_covid_fold3=valid_covid_folds[2]
new_valid_covid_fold4=valid_covid_folds[3]
new_valid_covid_fold5=valid_covid_folds[4]
#new_valid_covid_fold6=covid[covid['patient_id'].isin(valid_covid_folds[5][0])]
#new_valid_covid_fold7=covid[covid['patient_id'].isin(valid_covid_folds[6][0])]
#new_valid_covid_fold8=covid[covid['patient_id'].isin(valid_covid_folds[7][0])]
#new_valid_covid_fold9=covid[covid['patient_id'].isin(valid_covid_folds[8][0])]
#new_valid_covid_fold10=covid[covid['patient_id'].isin(valid_covid_folds[9][0])]

In [31]:
new_valid_normal_fold1=valid_normal_folds[0]
new_valid_normal_fold2=valid_normal_folds[1]
new_valid_normal_fold3=valid_normal_folds[2]
new_valid_normal_fold4=valid_normal_folds[3]
new_valid_normal_fold5=valid_normal_folds[4]
#new_valid_covid_fold6=covid[covid['patient_id'].isin(valid_covid_folds[5][0])]
#new_valid_covid_fold7=covid[covid['patient_id'].isin(valid_covid_folds[6][0])]
#new_valid_covid_fold8=covid[covid['patient_id'].isin(valid_covid_folds[7][0])]
#new_valid_covid_fold9=covid[covid['patient_id'].isin(valid_covid_folds[8][0])]
#new_valid_covid_fold10=covid[covid['patient_id'].isin(valid_covid_folds[9][0])]

In [32]:
new_train_covid_fold1=new_train_covid_fold1.reset_index()
new_train_covid_fold1=new_train_covid_fold1.drop(columns=['index'])
new_train_covid_fold2=new_train_covid_fold2.reset_index()
new_train_covid_fold2=new_train_covid_fold2.drop(columns=['index'])
new_train_covid_fold3=new_train_covid_fold3.reset_index()
new_train_covid_fold3=new_train_covid_fold3.drop(columns=['index'])
new_train_covid_fold4=new_train_covid_fold4.reset_index()
new_train_covid_fold4=new_train_covid_fold4.drop(columns=['index'])
new_train_covid_fold5=new_train_covid_fold5.reset_index()
new_train_covid_fold5=new_train_covid_fold5.drop(columns=['index'])
#new_train_covid_fold6=new_train_covid_fold6.reset_index()
#new_train_covid_fold6=new_train_covid_fold6.drop(columns=['index'])
#new_train_covid_fold7=new_train_covid_fold7.reset_index()
#new_train_covid_fold7=new_train_covid_fold7.drop(columns=['index'])
#new_train_covid_fold8=new_train_covid_fold8.reset_index()
#new_train_covid_fold8=new_train_covid_fold8.drop(columns=['index'])
#new_train_covid_fold9=new_train_covid_fold9.reset_index()
#new_train_covid_fold9=new_train_covid_fold9.drop(columns=['index'])
#new_train_covid_fold10=new_train_covid_fold10.reset_index()
#new_train_covid_fold10=new_train_covid_fold10.drop(columns=['index'])

In [33]:
new_train_normal_fold1=new_train_normal_fold1.reset_index()
new_train_normal_fold1=new_train_normal_fold1.drop(columns=['index'])
new_train_normal_fold2=new_train_normal_fold2.reset_index()
new_train_normal_fold2=new_train_normal_fold2.drop(columns=['index'])
new_train_normal_fold3=new_train_normal_fold3.reset_index()
new_train_normal_fold3=new_train_normal_fold3.drop(columns=['index'])
new_train_normal_fold4=new_train_normal_fold4.reset_index()
new_train_normal_fold4=new_train_normal_fold4.drop(columns=['index'])
new_train_normal_fold5=new_train_normal_fold5.reset_index()
new_train_normal_fold5=new_train_normal_fold5.drop(columns=['index'])
#new_train_normal_fold6=new_train_normal_fold6.reset_index()
#new_train_normal_fold6=new_train_normal_fold6.drop(columns=['index'])
#new_train_normal_fold7=new_train_normal_fold7.reset_index()
#new_train_normal_fold7=new_train_normal_fold7.drop(columns=['index'])
#new_train_normal_fold8=new_train_normal_fold8.reset_index()
#new_train_normal_fold8=new_train_normal_fold8.drop(columns=['index'])
#new_train_normal_fold9=new_train_normal_fold9.reset_index()
#new_train_normal_fold9=new_train_normal_fold9.drop(columns=['index'])
#new_train_normal_fold10=new_train_normal_fold10.reset_index()
#new_train_normal_fold10=new_train_normal_fold10.drop(columns=['index'])

In [34]:
new_valid_covid_fold1=new_valid_covid_fold1.reset_index()
new_valid_covid_fold1=new_valid_covid_fold1.drop(columns=['index'])
new_valid_covid_fold2=new_valid_covid_fold2.reset_index()
new_valid_covid_fold2=new_valid_covid_fold2.drop(columns=['index'])
new_valid_covid_fold3=new_valid_covid_fold3.reset_index()
new_valid_covid_fold3=new_valid_covid_fold3.drop(columns=['index'])
new_valid_covid_fold4=new_valid_covid_fold4.reset_index()
new_valid_covid_fold4=new_valid_covid_fold4.drop(columns=['index'])
new_valid_covid_fold5=new_valid_covid_fold5.reset_index()
new_valid_covid_fold5=new_valid_covid_fold5.drop(columns=['index'])
#new_valid_covid_fold6=new_valid_covid_fold6.reset_index()
#new_valid_covid_fold6=new_valid_covid_fold6.drop(columns=['index'])
#new_valid_covid_fold7=new_valid_covid_fold7.reset_index()
#new_valid_covid_fold7=new_valid_covid_fold7.drop(columns=['index'])
#new_valid_covid_fold8=new_valid_covid_fold8.reset_index()
#new_valid_covid_fold8=new_valid_covid_fold8.drop(columns=['index'])
#new_valid_covid_fold9=new_valid_covid_fold9.reset_index()
#new_valid_covid_fold9=new_valid_covid_fold9.drop(columns=['index'])
#new_valid_covid_fold10=new_valid_covid_fold10.reset_index()
#new_valid_covid_fold10=new_valid_covid_fold10.drop(columns=['index'])

In [35]:
new_valid_normal_fold1=new_valid_normal_fold1.reset_index()
new_valid_normal_fold1=new_valid_normal_fold1.drop(columns=['index'])
new_valid_normal_fold2=new_valid_normal_fold2.reset_index()
new_valid_normal_fold2=new_valid_normal_fold2.drop(columns=['index'])
new_valid_normal_fold3=new_valid_normal_fold3.reset_index()
new_valid_normal_fold3=new_valid_normal_fold3.drop(columns=['index'])
new_valid_normal_fold4=new_valid_normal_fold4.reset_index()
new_valid_normal_fold4=new_valid_normal_fold4.drop(columns=['index'])
new_valid_normal_fold5=new_valid_normal_fold5.reset_index()
new_valid_normal_fold5=new_valid_normal_fold5.drop(columns=['index'])
#new_valid_normal_fold6=new_valid_normal_fold6.reset_index()
#new_valid_normal_fold6=new_valid_normal_fold6.drop(columns=['index'])
#new_valid_normal_fold7=new_valid_normal_fold7.reset_index()
#new_valid_normal_fold7=new_valid_normal_fold7.drop(columns=['index'])
#new_valid_normal_fold8=new_valid_normal_fold8.reset_index()
#new_valid_normal_fold8=new_valid_normal_fold8.drop(columns=['index'])
#new_valid_normal_fold9=new_valid_normal_fold9.reset_index()
#new_valid_normal_fold9=new_valid_normal_fold9.drop(columns=['index'])
#new_valid_normal_fold10=new_valid_normal_fold10.reset_index()
#new_valid_normal_fold10=new_valid_normal_fold10.drop(columns=['index'])

In [52]:
def create_model(img_size,model_name,channels,classes,activation,weights,lr,crossentropy,optimizer_hyperparams,beta_1,beta_2,epsilon,decay,fine_tuning,unfrozen_layers):
    
    if model_name=='InceptionV3':
        base_model = tf.keras.applications.InceptionV3(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='InceptionResNetV2':
        base_model = tf.keras.applications.InceptionResNetV2(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='Xception':
        base_model = tf.keras.applications.Xception(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='DenseNet121':
        base_model = tf.keras.applications.DenseNet121(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='DenseNet169':
        base_model = tf.keras.applications.DenseNet169(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='DenseNet201':
        base_model = tf.keras.applications.DenseNet201(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='ResNet50':
        base_model = tf.keras.applications.ResNet50(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='ResNet101':
        base_model = tf.keras.applications.ResNet101(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='ResNet152':
        base_model = tf.keras.applications.ResNet152(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='MobileNetV2':
        base_model = tf.keras.applications.MobileNetV2(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
    elif model_name=='VGG16':
        base_model = tf.keras.applications.VGG16(include_top = False, weights = weights, input_shape = (img_size, img_size, channels))
        
    base_model.trainable=True
        
    x=base_model.output
    if model_name=='VGG16':
        x=tf.keras.layers.Flatten()(x)
    else:
        x=tf.keras.layers.GlobalAveragePooling2D()(x)
    #x=tf.keras.layers.Dropout(0.5)(x)
    out = tf.keras.layers.Dense(classes, activation = 'softmax')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = base_model.input, outputs = out)
    if optimizer_hyperparams==True:
        print('Optimizer Hyperparams: True')
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay)
    else:
        print('Optimizer Hyperparams: False')
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        
    model.compile(loss=crossentropy, optimizer=optimizer, metrics=['accuracy'])
    return model

In [53]:
def plot_confusion_matrix_val(cm, MODEL_NAME, foldno, CLASS1, CLASS2):
    
    with plt.style.context('default'):
        
        plt.figure()
    
        ax = sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME+' \n Fold '+str(foldno));
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_val_decimal'+str(foldno)+'.png')
        plt.show()
    
        ax = sns.heatmap(cm/np.sum(cm), annot=True, 
                fmt='.2%', cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME+' \n Fold '+str(foldno));
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_val_percentage'+str(foldno)+'.png')
        plt.show()
    
        group_counts = ["{0:0.0f}".format(value) for value in
                    cm.flatten()]
        group_percentages = ["{0:.2%}".format(value) for value in
                         cm.flatten()/np.sum(cm)]

        labels = [f"{v1}\n{v2}" for v1, v2 in
              zip(group_counts,group_percentages)]
        labels = np.asarray(labels).reshape(2,2)
        ax=sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME+' \n Fold '+str(foldno));
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_val_all'+str(foldno)+'.png')
        plt.show()

In [54]:
def plot_confusion_matrix_test(cm, MODEL_NAME, foldno, CLASS1, CLASS2):
    
    with plt.style.context('default'):
        
        plt.figure()
    
        ax = sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME+' \n Fold '+str(foldno));
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_test_decimal'+str(foldno)+'.png')
        plt.show()
    
        ax = sns.heatmap(cm/np.sum(cm), annot=True, 
                fmt='.2%', cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME+' \n Fold '+str(foldno));
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_test_percentage'+str(foldno)+'.png')
        plt.show()
    
        group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]
        group_percentages = ["{0:.2%}".format(value) for value in
                         cm.flatten()/np.sum(cm)]

        labels = [f"{v1}\n{v2}" for v1, v2 in
              zip(group_counts,group_percentages)]
        labels = np.asarray(labels).reshape(2,2)
        ax=sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME+' \n Fold '+str(foldno));
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_test_all'+str(foldno)+'.png')
        plt.show()

In [55]:
def plot_confusion_matrix_concat_val(val_total_cm, MODEL_NAME, foldno, CLASS1, CLASS2):
    
    with plt.style.context('default'):
        
        plt.figure()
    
        ax = sns.heatmap(val_total_cm, annot=True, fmt="d", cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME);
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_concat_val_decimal.png')
        plt.show()
    
        ax = sns.heatmap(val_total_cm/np.sum(val_total_cm), annot=True, 
            fmt='.2%', cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME);
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_concat_val_percentage.png')
        plt.show()
    
        group_counts = ["{0:0.0f}".format(value) for value in
                    val_total_cm.flatten()]
        group_percentages = ["{0:.2%}".format(value) for value in
                         val_total_cm.flatten()/np.sum(val_total_cm)]

        labels = [f"{v1}\n{v2}" for v1, v2 in
              zip(group_counts,group_percentages)]
        labels = np.asarray(labels).reshape(2,2)
        ax=sns.heatmap(val_total_cm, annot=labels, fmt='', cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME)
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_concat_val_all.png')
        plt.show()

In [56]:
def plot_confusion_matrix_concat_test(val_total_cm, MODEL_NAME, foldno, CLASS1, CLASS2):
    
    with plt.style.context('default'):
        
        plt.figure()
    
        ax = sns.heatmap(val_total_cm, annot=True, fmt="d", cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME);
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_concat_test_decimal.png')
        plt.show()
    
        ax = sns.heatmap(val_total_cm/np.sum(val_total_cm), annot=True, 
            fmt='.2%', cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME);
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_concat_test_percentage.png')
        plt.show()
    
        group_counts = ["{0:0.0f}".format(value) for value in
                    val_total_cm.flatten()]
        group_percentages = ["{0:.2%}".format(value) for value in
                         val_total_cm.flatten()/np.sum(val_total_cm)]

        labels = [f"{v1}\n{v2}" for v1, v2 in
              zip(group_counts,group_percentages)]
        labels = np.asarray(labels).reshape(2,2)
        ax=sns.heatmap(val_total_cm, annot=labels, fmt='', cmap='Blues')

        ax.set_title('Confusion Matrix for '+MODEL_NAME)
        ax.set_xlabel('\nPredicted')
        ax.set_ylabel('True');

        ## Ticket labels - List must be in alphabetical order
        ax.xaxis.set_ticklabels(['Covid-19','Normal'])
        ax.yaxis.set_ticklabels(['Covid-19','Normal'])

        ## Display the visualization of the Confusion Matrix.
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/confusion_matrix/cm_concat_test_all.png')
        plt.show()

In [57]:
def plot_acc_loss(history, MODEL_NAME, foldno):
    
    with plt.style.context('default'):
        
        plt.figure()
        # PLOT GRAPH BETWEEN TRAINING AND VALIDATION LOSS
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.legend(['Training Loss', 'Validation Loss'])
        plt.title("Training and validation Loss for "+MODEL_NAME+" \n Fold "+str(foldno))
        plt.xlabel('epoch')
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/loss_figure/loss_'+str(foldno)+'.png')
        plt.show()
        plt.close()
    
        # PLOT GRAPH BETWEEN TRAINING AND VALIDATION ACCURACY
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.legend(['Training Accuracy', 'Validation Accuracy'])
        plt.title("Training and Validation Accuracy for "+MODEL_NAME+" \n Fold "+str(foldno))
        plt.xlabel('epoch')
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/accuracy_figure/acc_'+str(foldno)+'.png')
        plt.show()
        plt.close()

In [58]:
class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        # store the model, the class index used to measure the class
        # activation map, and the layer to be used when visualizing
        # the class activation map
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName

        # if the layer name is None, attempt to automatically find
        # the target output layer
        if self.layerName is None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        # attempt to find the final convolutional layer in the network
        # by looping over the layers of the network in reverse order
        for layer in reversed(self.model.layers):
            # check to see if the layer has a 4D output
            if len(layer.output_shape) == 4:
                return layer.name

        # otherwise, we could not find a 4D layer so the GradCAM
        # algorithm cannot be applied
        raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")

    def compute_heatmap(self, image, eps=1e-8):
        # construct our gradient model by supplying (1) the inputs
        # to our pre-trained model, (2) the output of the (presumably)
        # final 4D layer in the network, and (3) the output of the
        # softmax activations from the model
        gradModel = Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output, 
                self.model.output])

        # record operations for automatic differentiation
        with tf.GradientTape() as tape:
            # cast the image tensor to a float-32 data type, pass the
            # image through the gradient model, and grab the loss
            # associated with the specific class index
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]

        # use automatic differentiation to compute the gradients
        grads = tape.gradient(loss, convOutputs)

        # compute the guided gradients
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads

        # the convolution and guided gradients have a batch dimension
        # (which we don't need) so let's grab the volume itself and
        # discard the batch
        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]

        # compute the average of the gradient values, and using them
        # as weights, compute the ponderation of the filters with
        # respect to the weights
        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        # grab the spatial dimensions of the input image and resize
        # the output class activation map to match the input image
        # dimensions
        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))

        # normalize the heatmap such that all values lie in the range
        # [0, 1], scale the resulting values to the range [0, 255],
        # and then convert to an unsigned 8-bit integer
        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")

        # return the resulting heatmap to the calling function
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5,
        colormap=cv2.COLORMAP_JET):
        # apply the supplied color map to the heatmap and then
        # overlay the heatmap on the input image
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)

        # return a 2-tuple of the color mapped heatmap and the output,
        # overlaid image
        return (heatmap, output)

In [59]:
def show_gradcam_covid_val(valid_data_covid, MODEL_NAME, foldno):
    
    with plt.style.context('default'):
        
        plt.figure()
        #index=0;
        for filename in valid_data_covid['image_path'][:100]:
            #index=index+1;
            #print(filename)
            last_part=filename.rpartition('/')[2]
            orig = cv2.imread(filename, cv2.IMREAD_COLOR)
            resized = cv2.resize(orig, (IMG_SIZE, IMG_SIZE))
            image = resized.astype(np.float64)
            image = np.expand_dims(image, axis=0)
            image = train_datagen.standardize(image)
    
            ground_truth = os.path.basename(os.path.dirname(filename))

            # use the network to make predictions on the input imag and find
            # the class label index with the largest corresponding probability
            preds = model.predict(image)
            i = np.argmax(preds[0])
            #print(str(np.argmax(preds)))
            # initialize our gradient class activation map and build the heatmap
            cam = GradCAM(model, i)
            heatmap = cam.compute_heatmap(image)
            #print(str(i))
            if(i==0):
                predicted='Covid-19'
            else:
                predicted='Normal'
            # resize the resulting heatmap to the original input image dimensions
            # and then overlay heatmap on top of the image
            heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
        
            (heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)
    
            plt.title('True: Covid-19  Predicted: '+predicted)
            plt.imshow(orig,cmap='gray',norm=mpl.colors.Normalize(vmin=0, vmax=255))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_orig_predicted_colorbar/comp_orig_predicted_colorbar'+str(foldno)+'_'+str(last_part))
            plt.close()
        
            plt.title('True: Covid-19  Predicted: '+predicted)
            plt.imshow(heatmap,cmap='jet_r',norm=mpl.colors.Normalize(vmin=0.0, vmax=1.0))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_heatmap_predicted_colorbar/comp_heatmap_predicted_colorbar'+str(foldno)+'_'+str(last_part))
            plt.close()
        
            plt.title('True: Covid-19  Predicted: '+predicted)
            plt.imshow(output,cmap='jet_r',norm=mpl.colors.Normalize(vmin=0.0, vmax=1.0))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_output_predicted_colorbar/comp_output_predicted_colorbar'+str(foldno)+'_'+str(last_part))
            plt.close()

In [60]:
def show_gradcam_normal_val(valid_data_normal, MODEL_NAME, foldno):
    
    with plt.style.context('default'):
        
        plt.figure()
        #index=0;
        for filename in valid_data_normal['image_path'][:100]:
            #index=index+1;
            #print(filename)
            last_part=filename.rpartition('/')[2]
            orig = cv2.imread(filename, cv2.IMREAD_COLOR)
            resized = cv2.resize(orig, (IMG_SIZE, IMG_SIZE))
            image = resized.astype(np.float64)
            image = np.expand_dims(image, axis=0)
            image = train_datagen.standardize(image)
    
            ground_truth = os.path.basename(os.path.dirname(filename))

            # use the network to make predictions on the input imag and find
            # the class label index with the largest corresponding probability
            preds = model.predict(image)
            i = np.argmax(preds[0])
            #print(str(np.argmax(preds)))
            # initialize our gradient class activation map and build the heatmap
            cam = GradCAM(model, i)
            heatmap = cam.compute_heatmap(image)
            #print(str(i))
            if(i==0):
                predicted='Covid-19'
            else:
                predicted='Normal'
            # resize the resulting heatmap to the original input image dimensions
            # and then overlay heatmap on top of the image
            heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
        
            (heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)
    
            plt.title('True: Normal  Predicted: '+predicted)
            plt.imshow(orig,cmap='gray',norm=mpl.colors.Normalize(vmin=0, vmax=255))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_orig_predicted_colorbar/comp_orig_predicted_colorbar'+str(foldno)+'_'+str(last_part))
            plt.close()

            plt.title('True: Normal  Predicted: '+predicted)
            plt.imshow(heatmap,cmap='jet_r',norm=mpl.colors.Normalize(vmin=0.0, vmax=1.0))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_heatmap_predicted_colorbar/comp_heatmap_predicted_colorbar'+str(foldno)+'_'+str(last_part))
            plt.close()

            plt.title('True: Normal  Predicted: '+predicted)
            plt.imshow(output,cmap='jet_r',norm=mpl.colors.Normalize(vmin=0.0, vmax=1.0))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_output_predicted_colorbar/comp_output_predicted_colorbar'+str(foldno)+'_'+str(last_part))
            plt.close()

In [61]:
def show_gradcam_covid_test(test_data_covid, MODEL_NAME, foldno):
    
    with plt.style.context('default'):
        
        plt.figure()
        #index=0;
        for filename in test_data_covid['image_path'][:100]:
            #index=index+1;
            #print(filename)
            last_part=filename.rpartition('/')[2]
            orig = cv2.imread(filename, cv2.IMREAD_COLOR)
            resized = cv2.resize(orig, (IMG_SIZE, IMG_SIZE))
            image = resized.astype(np.float64)
            image = np.expand_dims(image, axis=0)
            image = test_datagen.standardize(image)
    
            ground_truth = os.path.basename(os.path.dirname(filename))

            # use the network to make predictions on the input imag and find
            # the class label index with the largest corresponding probability
            preds = model.predict(image)
            i = np.argmax(preds[0])
            #print(str(np.argmax(preds)))
    
            # initialize our gradient class activation map and build the heatmap
            cam = GradCAM(model, i)
            heatmap = cam.compute_heatmap(image)
            #print(str(i))
            if(i==0):
                predicted='Covid-19'
            else:
                predicted='Normal'
            # resize the resulting heatmap to the original input image dimensions
            # and then overlay heatmap on top of the image
            heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
        
            (heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)
    
            plt.title('True: Covid-19  Predicted: '+predicted)
            plt.imshow(orig,cmap='gray',norm=mpl.colors.Normalize(vmin=0, vmax=255))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_orig_predicted_colorbar_test/comp_orig_predicted_colorbar_test'+str(foldno)+'_'+str(last_part))
            plt.close()
        
            plt.title('True: Covid-19  Predicted: '+predicted)
            plt.imshow(heatmap,cmap='jet_r',norm=mpl.colors.Normalize(vmin=0.0, vmax=1.0))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_heatmap_predicted_colorbar_test/comp_heatmap_predicted_colorbar_test'+str(foldno)+'_'+str(last_part))
            plt.close()
        
            plt.title('True: Covid-19  Predicted: '+predicted)
            plt.imshow(output,cmap='jet_r',norm=mpl.colors.Normalize(vmin=0.0, vmax=1.0))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_output_predicted_colorbar_test/comp_output_predicted_colorbar_test'+str(foldno)+'_'+str(last_part))
            plt.close()

In [62]:
def show_gradcam_normal_test(test_data_normal, MODEL_NAME, foldno):
    
    with plt.style.context('default'):
        
        plt.figure()
        #index=0;
        for filename in test_data_normal['image_path'][:100]:
            #index=index+1;
            #print(filename)
            last_part=filename.rpartition('/')[2]
            orig = cv2.imread(filename, cv2.IMREAD_COLOR)
            resized = cv2.resize(orig, (IMG_SIZE, IMG_SIZE))
            image = resized.astype(np.float64)
            image = np.expand_dims(image, axis=0)
            image = test_datagen.standardize(image)
    
            ground_truth = os.path.basename(os.path.dirname(filename))

            # use the network to make predictions on the input imag and find
            # the class label index with the largest corresponding probability
            preds = model.predict(image)
            i = np.argmax(preds[0])
            #print(str(np.argmax(preds)))
    
            # initialize our gradient class activation map and build the heatmap
            cam = GradCAM(model, i)
            heatmap = cam.compute_heatmap(image)
            #print(str(i))
            if(i==0):
                predicted='Covid-19'
            else:
                predicted='Normal'
            # resize the resulting heatmap to the original input image dimensions
            # and then overlay heatmap on top of the image
            heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
        
            (heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)
    
            plt.title('True: Normal  Predicted: '+predicted)
            plt.imshow(orig,cmap='gray',norm=mpl.colors.Normalize(vmin=0, vmax=255))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_orig_predicted_colorbar_test/comp_orig_predicted_colorbar_test'+str(foldno)+'_'+str(last_part))
            plt.close()
        
            plt.title('True: Normal  Predicted: '+predicted)
            plt.imshow(heatmap,cmap='jet_r',norm=mpl.colors.Normalize(vmin=0.0, vmax=1.0))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_heatmap_predicted_colorbar_test/comp_heatmap_predicted_colorbar_test'+str(foldno)+'_'+str(last_part))
            plt.close()
        
            plt.title('True: Normal  Predicted: '+predicted)
            plt.imshow(output,cmap='jet_r',norm=mpl.colors.Normalize(vmin=0.0, vmax=1.0))
            plt.colorbar()
            plt.savefig('/kaggle/working/'+MODEL_NAME+'/comp_output_predicted_colorbar_test/comp_output_predicted_colorbar_test'+str(foldno)+'_'+str(last_part))
            plt.close()

In [63]:
def plot_roc_curve_val_probs(fpr,tpr,foldno,roc_auc,MODEL_NAME):
    
    with plt.style.context('default'):
    
        plt.figure()
        plt.plot(fpr, tpr, lw=2, alpha=0.8, color='b', label='ROC fold %d (AUC = %0.2f)' % (foldno, roc_auc))
        plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'r',label="Chance",alpha=0.8)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic for '+MODEL_NAME+" \n Fold "+str(foldno))
        plt.legend(loc="lower right")
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/roc_curve_val/validation_roc_curve_probs'+str(foldno)+'.png')
        plt.show()
        plt.close()

In [64]:
def plot_roc_curve_val_labels(fpr,tpr,foldno,roc_auc,MODEL_NAME):
    
    with plt.style.context('default'):
    
        plt.figure()
        plt.plot(fpr, tpr, lw=2, alpha=0.8, color='b', label='ROC fold %d (AUC = %0.2f)' % (foldno, roc_auc))
        plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'r',label="Chance",alpha=0.8)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic for '+MODEL_NAME+" \n Fold "+str(foldno))
        plt.legend(loc="lower right")
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/roc_curve_val/validation_roc_curve_labels'+str(foldno)+'.png')
        plt.show()
        plt.close()

In [65]:
def plot_roc_curve_test(fpr,tpr,foldno,roc_auc,MODEL_NAME):
    
    with plt.style.context('default'):
    
        plt.figure()
        plt.plot(fpr, tpr, lw=2, alpha=0.8, color='b', label='ROC fold %d (AUC = %0.2f)' % (foldno, roc_auc))
        plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'r',label="Chance",alpha=0.8)
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic for '+MODEL_NAME+" \n Fold "+str(foldno))
        plt.legend(loc="lower right")
        plt.savefig('/kaggle/working/'+MODEL_NAME+'/roc_curve_test/test_roc_curve_'+str(foldno)+'.png')
        plt.show()
        plt.close()

In [66]:
if ANNEALER==True:
    print('Annealer: True')
    annealer = keras.callbacks.ReduceLROnPlateau(monitor=ANNEALER_MONITOR, factor=ANNEALER_FACTOR, patience=ANNEALER_PATIENCE, verbose=1)
if EARLY==True:
    print('Early: True')
    early = keras.callbacks.EarlyStopping(monitor=EARLY_MONITOR, patience=EARLY_PATIENCE,verbose=1)

In [67]:
# CREATING TRAINING, TESTING AND VALIDATION BATCHES

if AUGMENTATION==True:
    print("Augmentation: True")
    train_datagen = ImageDataGenerator(#preprocessing_function=preprocess_input,
                                    rescale=1./255,
                                    #samplewise_center = SAMPLEWISE_CENTER,
                                    #samplewise_std_normalization = SAMPLEWISE_STD_NORM,
                                    rotation_range=ROTATION_RANGE,
                                    width_shift_range=WIDTH_SHIFT_RANGE,
                                    height_shift_range=HEIGHT_SHIFT_RANGE,
                                    #zoom_range=ZOOM_RANGE,
                                    horizontal_flip=HORIZONTAL_FLIP,
                                    vertical_flip=VERTICAL_FLIP)
else:
    print("Augmentation: False")
    train_datagen = ImageDataGenerator(#preprocessing_function=preprocess_input)
                                    rescale=1./255)
                                    #samplewise_center = SAMPLEWISE_CENTER,
                                    #samplewise_std_normalization = SAMPLEWISE_STD_NORM)

val_datagen = ImageDataGenerator(#preprocessing_function=preprocess_input)
                                    rescale=1./255)
                                    #samplewise_center = SAMPLEWISE_CENTER,
                                    #samplewise_std_normalization = SAMPLEWISE_STD_NORM)

test_datagen = ImageDataGenerator(#preprocessing_function=preprocess_input)
                                    rescale=1./255)
                                    #samplewise_center = SAMPLEWISE_CENTER,
                                    #samplewise_std_normalization = SAMPLEWISE_STD_NORM)

Augmentation: True


In [ ]:
#test_covid_df=test_covid_df.reset_index()
#test_covid_df=test_covid_df.drop(columns=['index'])
#test_normal_df=test_normal_df.reset_index()
#test_normal_df=test_normal_df.drop(columns=['index'])

In [ ]:
#test=pd.concat([test_covid_df,test_normal_df])
#test=test.drop(columns='patient_id')
#test=test.reset_index()
#test=test.drop(columns=['index'])

In [ ]:
#test['images']=test["images"].str[:-3]
#test['images']=test['images']+'tif'

In [ ]:
#test_generator = test_datagen.flow_from_dataframe(test,
#                                                x_col="images",
#                                                y_col="target",
#                                               target_size = (IMG_SIZE, IMG_SIZE), 
#                                               batch_size = BATCH_SIZE,
#                                               shuffle=False,
#                                               class_mode = CLASS_MODE)

In [ ]:
print(FOLDNO)
    
if FOLDNO==1:
    train_fold=pd.concat([new_train_covid_fold1,new_train_normal_fold1])
    valid_fold=pd.concat([new_valid_covid_fold1,new_valid_normal_fold1])
elif FOLDNO==2:
    train_fold=pd.concat([new_train_covid_fold2,new_train_normal_fold2])
    valid_fold=pd.concat([new_valid_covid_fold2,new_valid_normal_fold2])
elif FOLDNO==3:
    train_fold=pd.concat([new_train_covid_fold3,new_train_normal_fold3])
    valid_fold=pd.concat([new_valid_covid_fold3,new_valid_normal_fold3])
elif FOLDNO==4:
    train_fold=pd.concat([new_train_covid_fold4,new_train_normal_fold4])
    valid_fold=pd.concat([new_valid_covid_fold4,new_valid_normal_fold4])
elif FOLDNO==5:
    train_fold=pd.concat([new_train_covid_fold5,new_train_normal_fold5])
    valid_fold=pd.concat([new_valid_covid_fold5,new_valid_normal_fold5])
        
train_fold=train_fold.reset_index()
train_fold=train_fold.drop(columns=['index'])
valid_fold=valid_fold.reset_index()
valid_fold=valid_fold.drop(columns=['index'])

    #train_fold['images']=train_fold["images"].str[:-3]
    #train_fold['images']=train_fold['images']+'tif'

    #valid_fold['images']=valid_fold["images"].str[:-3]
    #valid_fold['images']=valid_fold['images']+'tif'

train_generator = train_datagen.flow_from_dataframe(train_fold,
                                                        x_col="image_path",
                                                        y_col="target",
                                                         target_size = (IMG_SIZE, IMG_SIZE),
                                                         #subset = 'training',
                                                         #interpolation="bilinear",
                                                         batch_size = BATCH_SIZE,
                                                         shuffle=True,
                                                         class_mode = CLASS_MODE)

validation_generator = val_datagen.flow_from_dataframe(valid_fold,
                                                          x_col="image_path",
                                                          y_col="target",
                                                          target_size = (IMG_SIZE, IMG_SIZE), 
                                                          #subset = "validation"
                                                          #interpolation="bilinear",
                                                          batch_size = BATCH_SIZE,
                                                          shuffle=False,
                                                          class_mode = CLASS_MODE)
    
os.mkdir('/kaggle/working/'+MODEL_NAME)
os.mkdir('/kaggle/working/'+MODEL_NAME+'/comp_orig_predicted_colorbar')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/comp_heatmap_predicted_colorbar')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/comp_output_predicted_colorbar')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/confusion_matrix')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/loss_figure')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/accuracy_figure')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/metrics')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/models')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/roc_curve_val')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/roc_curve_test')
#os.mkdir('/kaggle/working/'+MODEL_NAME+'/roc_auc_curve')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/comp_orig_predicted_colorbar_test')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/comp_heatmap_predicted_colorbar_test')
os.mkdir('/kaggle/working/'+MODEL_NAME+'/comp_output_predicted_colorbar_test')

checkpoint=tf.keras.callbacks.ModelCheckpoint('/kaggle/working/'+MODEL_NAME+'/models/model_'+str(FOLDNO)+'.h5',save_best_only=True,verbose=1)

model=create_model(IMG_SIZE,MODEL_NAME,CHANNELS,CLASSES,ACTIVATION,WEIGHTS,LEARNING_RATE,CROSSENTROPY,OPTIMIZER_HYPERPARAMS,BETA_1,BETA_2,EPSILON,DECAY,FINE_TUNING,UNFROZEN_LAYERS)

model.summary()

start = time.time()
if ANNEALER==True and EARLY==True:
    history = model.fit(train_generator, 
                            epochs = EPOCHS, 
                            validation_data = validation_generator,
                            callbacks=[annealer, early, checkpoint])
elif ANNEALER==True and EARLY==False:
    history = model.fit(train_generator, 
                            epochs = EPOCHS, 
                            validation_data = validation_generator,
                            callbacks=[annealer, checkpoint])
elif ANNEALER==False and EARLY==True:
    history = model.fit(train_generator, 
                            epochs = EPOCHS, 
                            validation_data = validation_generator,
                            callbacks=[early, checkpoint])
else:
    history = model.fit(train_generator, 
                            epochs = EPOCHS, 
                            validation_data = validation_generator,
                            callbacks=[checkpoint])
end = time.time()
final_time=end-start

plot_acc_loss(history, MODEL_NAME, FOLDNO)

history_fold=history.history
    
print(history.history['accuracy'][-1])
print(history.history['loss'][-1])
    
train_acc_all=[]
train_loss_all=[]
train_acc_all.append(str(history.history['accuracy'][-1]))
train_loss_all.append(str(history.history['loss'][-1]))

model.load_weights('/kaggle/working/'+MODEL_NAME+'/models/model_'+str(FOLDNO)+'.h5')

val_loss, val_acc = model.evaluate(validation_generator)
print("Validation Accuracy: "+str(val_acc))
print("Validation Loss: "+str(val_loss))
    
val_acc_all=[]
val_loss_all=[]
val_acc_all.append(str(val_acc))
val_loss_all.append(str(val_loss))

target_names = []

for key in validation_generator.class_indices:

    target_names.append(key)
    
Y_pred = model.predict(validation_generator)
        
y_pred = np.argmax(Y_pred, axis=1)

np.set_printoptions(threshold=np.inf)
prob=model.predict(validation_generator)[:,1]

val_total_cm=[]
cm = confusion_matrix(validation_generator.classes, y_pred)
val_total_cm.append(cm)

plot_confusion_matrix_val(cm,MODEL_NAME, FOLDNO, CLASS1, CLASS2)

specificity = cm[1,1]/(cm[1,0]+cm[1,1])
    
print("Specificity: "+str(specificity))

print(classification_report(validation_generator.classes, y_pred, target_names=target_names))
    
precision,recall,f1score,support=score(validation_generator.classes,y_pred,average=None)
precision_micro,recall_micro,f1score_micro,support_micro=score(validation_generator.classes,y_pred,average='micro')
precision_macro,recall_macro,f1score_macro,support_macro=score(validation_generator.classes,y_pred,average='macro')
precision_weighted,recall_weighted,f1score_weighted,support_weighted=score(validation_generator.classes,y_pred,average='weighted')
    
tprs_probs = []
aucs_probs = []
mean_fpr_probs = np.linspace(0,1,100)

fpr_probs, tpr_probs, t_probs = roc_curve(validation_generator.classes, prob)
tprs_probs.append(interp(mean_fpr_probs, fpr_probs, tpr_probs))
roc_auc_probs = auc(fpr_probs, tpr_probs)
aucs_probs.append(roc_auc_probs)

with open('/kaggle/working/'+MODEL_NAME+'/fpr_probs_fold'+str(FOLDNO)+'.txt', 'w') as f:
    for elem in fpr_probs:
        f.write(str(elem))
        f.write('\n')

with open('/kaggle/working/'+MODEL_NAME+'/tpr_probs_fold'+str(FOLDNO)+'.txt', 'w') as f:
    for elem in tpr_probs:
        f.write(str(elem))
        f.write('\n')

with open('/kaggle/working/'+MODEL_NAME+'/t_probs_fold'+str(FOLDNO)+'.txt', 'w') as f:
    for elem in t_probs:
        f.write(str(elem))
        f.write('\n')
        
plot_roc_curve_val_probs(fpr_probs,tpr_probs,FOLDNO,roc_auc_probs,MODEL_NAME)

tprs_labels = []
aucs_labels = []
mean_fpr_labels = np.linspace(0,1,100)

fpr_labels, tpr_labels, t_labels = roc_curve(validation_generator.classes, y_pred)
tprs_labels.append(interp(mean_fpr_labels, fpr_labels, tpr_labels))
roc_auc_labels = auc(fpr_labels, tpr_labels)
aucs_labels.append(roc_auc_labels)

with open('/kaggle/working/'+MODEL_NAME+'/fpr_labels_fold'+str(FOLDNO)+'.txt', 'w') as f:
    for elem in fpr_labels:
        f.write(str(elem))
        f.write('\n')

with open('/kaggle/working/'+MODEL_NAME+'/tpr_labels_fold'+str(FOLDNO)+'.txt', 'w') as f:
    for elem in tpr_labels:
        f.write(str(elem))
        f.write('\n')

with open('/kaggle/working/'+MODEL_NAME+'/t_labels_fold'+str(FOLDNO)+'.txt', 'w') as f:
    for elem in t_labels:
        f.write(str(elem))
        f.write('\n')
        
plot_roc_curve_val_labels(fpr_labels,tpr_labels,FOLDNO,roc_auc_labels,MODEL_NAME)

valid_covid=valid_fold[valid_fold.target == 'Covid-19']
valid_covid=valid_covid.sample(frac=1,random_state=SEED)
print(valid_covid)
valid_normal=valid_fold[valid_fold.target == 'Normal']
valid_normal=valid_normal.sample(frac=1,random_state=SEED)
print(valid_normal)

show_gradcam_covid_val(valid_covid, MODEL_NAME, FOLDNO)

show_gradcam_normal_val(valid_normal, MODEL_NAME, FOLDNO)

data1=[]
data1.append([FOLDNO,train_acc_all[0],train_loss_all[0],val_acc_all[0],val_loss_all[0],precision[0],precision[1],precision_micro,precision_macro,precision_weighted,recall[0],recall[1],recall_micro,recall_macro,recall_weighted,f1score[0],f1score[1],f1score_micro,f1score_macro,f1score_weighted,specificity,support[0],support[1],val_total_cm[0][0][0],val_total_cm[0][0][1],val_total_cm[0][1][0],val_total_cm[0][1][1],final_time])

validation_metrics = pd.DataFrame(data1, columns = ['Fold','Training Accuracy', 'Training Loss','Validation Accuracy', 'Validation Loss', 'Precision Covid-19', 'Precision Viral Pneumonia', 'Precision Micro','Precision Macro','Precision Weighted','Recall Covid-19','Recall Viral Pneumonia','Recall Micro','Recall Macro','Recall Weighted','F1-Score Covid-19','F1-Score Viral Pneumonia','F1-Score Micro','F1-Score Macro','F1-Score Weighted','Specificity','Support Covid-19','Support Viral Pneumonia','[0,0]','[0,1]','[1,0]','[1,1]','Time'])

validation_metrics.to_excel("/kaggle/working/"+MODEL_NAME+"/metrics/validation_metrics.xlsx",
            sheet_name='Sheet1')

data2=[]
data2.append([N_SPLITS,IMG_SIZE,CHANNELS,LEARNING_RATE,POOLING,BATCH_SIZE,TEST_SIZE,CLASSES,CLASS1,CLASS2,FINE_TUNING,UNFROZEN_LAYERS,EPOCHS,CLASS_MODE,CROSSENTROPY,MODEL_NAME,OPTIMIZER,OPTIMIZER_HYPERPARAMS,BETA_1,BETA_2,EPSILON,DECAY,ANNEALER,ANNEALER_MONITOR,ANNEALER_FACTOR,ANNEALER_PATIENCE,EARLY,EARLY_MONITOR,EARLY_PATIENCE,RESCALE,SAMPLEWISE_CENTER,SAMPLEWISE_STD_NORM,AUGMENTATION,ROTATION_RANGE,WIDTH_SHIFT_RANGE,HEIGHT_SHIFT_RANGE,ZOOM_RANGE,HORIZONTAL_FLIP,VERTICAL_FLIP])

hyperparameters = pd.DataFrame(data2, columns = ['Number of Folds', 'Image Size', 'Channels','Initial LR','Pooling','Batch Size','Test Size','Classes','Class 1','Class 2','Fine Tuning','Unfrozen Layers','Epochs','Class Mode','Crossentropy','Model Name','Optimizer','Optimizer Hyperparameters','Beta 1','Beta 2','Epsilon','Decay','Implements Annealer','Annealer Monitor','Annealer Factor','Annealer Patience','Implements Early Stopping','Early Stopping Monitor','Early Stopping Patience','Rescale','Samplewise Center','Samplewise Std Normalization','Implements Augmentation','Rotation Range','Width Shift Range','Height Shift Range','Zoom Range','Horizontal Flip','Vertical Flip'])

hyperparameters.to_excel("/kaggle/working/"+MODEL_NAME+"/metrics/hyperparameters.xlsx",
            sheet_name='Sheet1') 

history_df_fold = pd.DataFrame(history_fold)

history_df_fold.to_excel("/kaggle/working/"+MODEL_NAME+"/metrics/history_df_fold.xlsx",
            sheet_name='Sheet1')
    
shutil.make_archive(str(MODEL_NAME)+"_"+str(ROTATION_RANGE)+"_fold_"+str(FOLDNO)+"_"+str(NORMALIZED)+"_"+str(SPLIT)+"_"+str(SEGMENTED), 'zip', "/kaggle/working/"+str(MODEL_NAME))

1


In [ ]:
print('AAAA')